In [2]:
import bw2data as bd
import bw2calc as bc
import bw2io as bi
from pathlib import Path
from gwp_uncertainties import add_bw_method_with_gwp_uncertainties

# Local files
from consumption_model_ch.import_databases import (
    import_exiobase_3,
    import_consumption_db,
)
from consumption_model_ch.consumption_fus import (
    add_consumption_activities,
    
    add_consumption_categories,
    add_consumption_sectors,
    add_archetypes_consumption,
    get_archetypes_scores_per_month,
    get_archetypes_scores_per_sector,
)
from consumption_model_ch.plot_archetypes import plot_archetypes_scores_yearly, plot_archetypes_scores_per_sector

# Set environment variables for the bentso package
%set_env ENTSOE_API_TOKEN=98eb7887-8d16-43f4-b75c-1e8a80be520e
%set_env BENTSO_DATA_DIR=/Users/akim/Documents/LCA_files/entso-data-cache

import sys
sys.path.append('/Users/akim/PycharmProjects/akula')
from akula.electricity import replace_ei_with_entso, add_swiss_residual_mix

env: ENTSOE_API_TOKEN=98eb7887-8d16-43f4-b75c-1e8a80be520e
env: BENTSO_DATA_DIR=/Users/akim/Documents/LCA_files/entso-data-cache
Using data directory /Users/akim/Documents/LCA_files/entso-data-cache


In [ ]:
def import_all_databases(use_exiobase, year='091011', add_activities=True, add_archetypes=False):

    path_base = Path('/Users/akim/Documents/LCA_files/')

    directory_habe = path_base / 'HABE_2017/'
    fp_ecoinvent_38 = path_base / "ecoinvent_38_cutoff" / "datasets"
    # fp_ecoinvent_38 = "/Users/cmutel/Documents/lca/Ecoinvent/3.8/cutoff/datasets"
    fp_ecoinvent_33 = path_base / "ecoinvent_33_cutoff"/ "datasets"
    fp_exiobase = path_base / "exiobase_381_monetary" / "IOT_2015_pxp"
    # fp_archetypes = path_base / "heia" / "hh_archetypes_weighted_ipcc_091011.csv"

    ei38_name = "ecoinvent 3.8 cutoff"
    ex38_name = "exiobase 3.8.1 monetary"
    co_name = "swiss consumption 1.0"

    if use_exiobase:
        project = "GSA for archetypes with exiobase"
    else:
        project = "GSA for archetypes"
    bd.projects.set_current(project)

    # Import biosphere and ecoinvent databases
    if ei38_name not in bd.databases:
        bi.bw2setup()
        ei = bi.SingleOutputEcospold2Importer(fp_ecoinvent_38, ei38_name)
        ei.apply_strategies()
        assert ei.all_linked
        ei.write_database()

    exclude_databases = [
        'heia',
        'Agribalyse 1.2',
        'Agribalyse 1.3 - {}'.format(ei38_name),
    ]

    # Import exiobase
    if use_exiobase:
        import_exiobase_3(fp_exiobase, ex38_name)
    else:
        exclude_databases.append('exiobase 2.2')

    # Import consumption database
    try:
        del bd.databases["swiss consumption 1.0"]
    except:
        pass
    if co_name not in bd.databases:
        import_consumption_db(
            directory_habe, co_name, fp_ecoinvent_33, exclude_databases, fp_exiobase,
        )
        
    # Modify electricity mixes
    add_swiss_residual_mix()
    replace_ei_with_entso()

    # Add uncertainties to GWP values
    method = ("IPCC 2013", "climate change", "GWP 100a", "uncertain")
    if method not in bd.methods:
        add_bw_method_with_gwp_uncertainties()

    # Add functional units
    co = bd.Database(co_name)
    option = 'aggregated'
    if add_activities:
        add_consumption_activities(co_name, year, option=option,)
        add_consumption_categories(co_name)
        add_consumption_sectors(co_name, year)
        if add_archetypes:
            add_archetypes_consumption(co_name, year)

    # LCIA for average consumption
    co_average_act_name = f'ch hh average consumption {option}, years {year}'
    hh_average = [act for act in co if co_average_act_name == act['name']]
    assert len(hh_average) == 1
    demand_act = hh_average[0]
    lca = bc.LCA({demand_act: 1}, method)
    lca.lci()
    lca.lcia()
    print("{:8.3f}  {}".format(lca.score, demand_act['name']))

    # LCIA for all Swiss consumption sectors
    sectors = sorted([act for act in co if "sector" in act['name'].lower() and str(year) in act['name']])
    for demand_act in sectors:
        lca = bc.LCA({demand_act: 1}, method)
        lca.lci()
        lca.lcia()
        print("{:8.3f}  {}".format(lca.score, demand_act['name']))
        

def plot_archetypes(use_exiobase, year):
    # Add archetypes and compute total yearly scores per archetype
    if use_exiobase:
        project = "GSA for archetypes with exiobase"
    else:
        project = "GSA for archetypes"
    bd.projects.set_current(project)
    co_name = "swiss consumption 1.0"
    write_dir = Path("write_files") / project.lower().replace(" ", "_") / "archetype_scores"
    write_dir.mkdir(parents=True, exist_ok=True)

    method = ("IPCC 2013", "climate change", "GWP 100a", "uncertain")
    fp_archetypes_scores = write_dir / f"monthly_scores_{year}.pickle"
    archetypes_scores_monthly = get_archetypes_scores_per_month(co_name, year, method, fp_archetypes_scores)

    # Compare with Andi's results (reproduce Fig. 3 in Andi's data mining paper, top part)
    num_months_in_year = 12
    archetypes_scores_yearly = {
        archetype: score*num_months_in_year for archetype, score in archetypes_scores_monthly.items()
    }
    fig = plot_archetypes_scores_yearly(archetypes_scores_yearly)
    fig.write_html( write_dir / f"yearly_scores_{year}.html")
    fig.write_image(write_dir / f"yearly_scores_{year}.pdf")
    fig.show()

    # Compare with Andi's contributions per sectors (reproduce Fig. 3 in Andi's data mining paper, bottom part)
    fp_archetypes_scores_sectors = write_dir / f"monthly_scores_{year}.pickle"
    archetypes_scores_per_sector = get_archetypes_scores_per_sector(co_name, year, method, write_dir)
    fig = plot_archetypes_scores_per_sector(archetypes_scores_per_sector, year)
    fig.write_html(write_dir / f"sector_scores_{year}.html")
    fig.write_image(write_dir / f"sector_scores_{year}.pdf")
    fig.show()


if __name__ == "__main__":

    print("Impacts WITHOUT exiobase")
    print("------------------------")
    use_ex = False
    year = '151617'
    import_all_databases(use_ex, year, True, False)
#     plot_archetypes(use_ex, year)

    print("\n")
    print("Impacts WITH exiobase")
    print("---------------------")
    use_ex = True
    import_all_databases(use_ex, year, True, False)
    plot_archetypes(use_ex, year)


In [37]:
bd.projects.set_current("GSA for archetypes")
co = bd.Database('swiss consumption 1.0')
fu = [act for act in co if "ch hh average consumption aggregated, years 151617" == act['name']][0]
demand = {fu: 1}
method = ("IPCC 2013", "climate change", "GWP 100a", "uncertain")

fu_mapped, packages, _ = bd.prepare_lca_inputs(demand=demand, method=method, remapping=False)

lca = bc.LCA(demand=fu_mapped, data_objs=packages, use_distributions=False)
lca.lci()
lca.lcia()
lca.score

1135.223913835494

In [38]:
from pypardiso import spsolve
A = lca.technosphere_matrix
B = lca.biosphere_matrix
C = lca.characterization_matrix
d = lca.demand_array

In [47]:
# trow, tcol = lca.dicts.activity[15273], lca.dicts.activity[21608]
# brow, bcol = lca.dicts.biosphere[1226], lca.dicts.activity[21608]
trow, tcol = lca.dicts.activity[21877], lca.dicts.activity[8082]
brow, bcol = lca.dicts.biosphere[1229], lca.dicts.activity[8082]

In [48]:
trow, tcol, brow, bcol

(17449, 3654, 724, 3654)

In [40]:
A[lca.dicts.activity[15273], lca.dicts.activity[21608]]

-22.6299991607666

In [41]:
B[lca.dicts.biosphere[1226], lca.dicts.activity[21608]]

70.38655090332031

In [45]:
A[trow, tcol], B[brow, bcol]

(-2.26299991607666, 7.038655090332031)

In [42]:
A[trow, tcol] /= 10
B[brow, bcol] /= 10

In [18]:
from gsa_framework.utils import read_pickle
data = read_pickle("combustion_data.pickle")

vals = []
for i,element in enumerate(data["bindices"]):
    row, col = lca.dicts.biosphere[element['row']], lca.dicts.activity[element['col']]
    vals.append(B[row, col] / data['bdata'][i, 0])
#     print(B[row, col] / data['bdata'][i, 0])
#     B[element] = data['bdata'][i, 0]

In [43]:
score = sum(C*B*spsolve(A, d))
score

1135.2239133556093

In [ ]:
lca.score

In [ ]:
1135.2239133556093-1135.2239186343616

In [ ]:
data[0,0]

In [ ]:
data['indices_bio']

In [ ]:
B[lca.dicts.biosphere[1229], lca.dicts.activity[8082]]

In [29]:
data['tdata'][0,0]

226.29999999999998

In [54]:
data['indices_tech']

[array([(15273, 21608)], dtype=[('row', '<i4'), ('col', '<i4')]),
 array([(21877, 8082), ( 6596, 8082)],
       dtype=[('row', '<i4'), ('col', '<i4')]),
 array([( 7387, 13720), (11220, 13720), (21240, 13720), ( 6236, 13720),
        (15273, 13720)], dtype=[('row', '<i4'), ('col', '<i4')]),
 array([(6236, 12785), (6744, 12785)],
       dtype=[('row', '<i4'), ('col', '<i4')]),
 array([(23542, 21073)], dtype=[('row', '<i4'), ('col', '<i4')]),
 array([(21877, 19799)], dtype=[('row', '<i4'), ('col', '<i4')]),
 array([(11220, 7083), ( 6236, 7083), (15273, 7083)],
       dtype=[('row', '<i4'), ('col', '<i4')]),
 array([( 8890, 11978), (21775, 11978), (20914, 11978), ( 8518, 11978),
        ( 5297, 11978), (12709, 11978), ( 4728, 11978)],
       dtype=[('row', '<i4'), ('col', '<i4')]),
 array([( 7427, 18168), ( 7387, 18168), (11220, 18168), (21240, 18168),
        ( 6236, 18168), (15273, 18168)],
       dtype=[('row', '<i4'), ('col', '<i4')]),
 array([(15273, 12849)], dtype=[('row', '<i4'), ('

In [51]:
data['sample_bio'], 

([array([[703.86548043]]),
  array([[6.91744148, 4.09762158]]),
  array([[  1.13875101,  47.80658296,   3.34865559,   4.84008659,
          472.06592385]]),
  array([[0.78890042, 0.29568426]]),
  array([[0.167]]),
  array([[0.38032819]]),
  array([[0.14219148, 0.01439591, 1.40406923]]),
  array([[0.44526481, 0.01976508, 0.08168475, 0.01970088, 0.0737009 ,
          0.00405822, 2.26847085]]),
  array([[0.26219962, 0.01260502, 0.5291788 , 0.03706681, 0.0535757 ,
          5.22537405]]),
  array([[0.73912062]]),
  array([[2.6484654 , 0.05047924]]),
  array([[1.10958013e-05, 3.01623114e-05, 1.26626192e-03, 8.86964681e-05,
          1.28200280e-04, 1.25036986e-02]]),
  array([[19.47027248,  0.93601644, 81.76736784, 39.29546245,  2.75248641,
           3.97839438]]),
  array([[8.3834464]]),
  array([[0.07476496, 0.07547681, 0.15090338, 0.00500585, 0.03386047,
          1.80187145]]),
  array([[2.00746286]]),
  array([[ 2.88737404,  0.138808  , 12.12581771,  5.8273811 ,  0.40818421,
         

In [49]:
data['indices_bio']

[array([(1226, 21608)], dtype=[('row', '<i4'), ('col', '<i4')]),
 array([(1229, 8082)], dtype=[('row', '<i4'), ('col', '<i4')]),
 array([(1226, 13720)], dtype=[('row', '<i4'), ('col', '<i4')]),
 array([(1229, 12785)], dtype=[('row', '<i4'), ('col', '<i4')]),
 array([(1226, 21073)], dtype=[('row', '<i4'), ('col', '<i4')]),
 array([(1229, 19799)], dtype=[('row', '<i4'), ('col', '<i4')]),
 array([(118, 7083)], dtype=[('row', '<i4'), ('col', '<i4')]),
 array([(1229, 11978)], dtype=[('row', '<i4'), ('col', '<i4')]),
 array([(1229, 18168)], dtype=[('row', '<i4'), ('col', '<i4')]),
 array([(1229, 12849)], dtype=[('row', '<i4'), ('col', '<i4')]),
 array([(118, 17585)], dtype=[('row', '<i4'), ('col', '<i4')]),
 array([(1226, 6224)], dtype=[('row', '<i4'), ('col', '<i4')]),
 array([(1226, 8070)], dtype=[('row', '<i4'), ('col', '<i4')]),
 array([(1229, 20331)], dtype=[('row', '<i4'), ('col', '<i4')]),
 array([(118, 11198)], dtype=[('row', '<i4'), ('col', '<i4')]),
 array([(1226, 7646)], dtype=[('

In [ ]:
%%time 
# Backup GSA project
bi.backup_project_directory("GSA for archetypes")
# bi.backup_project_directory("GSA for archetypes with exiobase")
# # Restore GSA project
# fp_gsa_project = path_base / "brightway2-project-GSA-backup.16-November-2021-11-50AM.tar.gz"
# if project not in bd.projects:
#     bi.restore_project_directory(fp_gsa_project)